In [64]:
from transpose import Transpose
from dateutil import parser
from datetime import datetime, timedelta

transpose = Transpose('3OGpVlBjcr5OabDWF3lDuzzDqNUtHiDt')

In [2]:
# specify a target account (ENS or address)
target_account = '0x5eeeFc7D872C03296639bcc49D11c99403d2F4A8'

In [14]:
# STEP 1: pull all owned NFTs
owned_nfts = transpose.nft.nfts_by_owner(owner_address=target_account, limit=500)
owned_nfts = owned_nfts.to_dict()

assert len(owned_nfts) > 0, 'No NFTs owned!'

In [12]:
# STEP 2: pull all mints, sends, and burns from last month

# get date from exactly a month ago
month_ago_dt = datetime.now() - timedelta(days=30)

# pull all nft activity
nft_transfers = transpose.nft.transfers_by_account(
    account_address=target_account, 
    transferred_after=month_ago_dt, 
    limit=500
)

nft_transfers = nft_transfers.to_dict()

In [13]:
# STEP 3: pull all sales from the last month
nft_sales = transpose.nft.sales_by_account(
  account_address=target_account, 
  sold_after=month_ago_dt, 
  limit=500
)

nft_sales = nft_sales.to_dict()

In [73]:
# STEP 4: aggregate nft activity

_target_account = owned_nfts[0]['owner']
nft_activity_map = {}

# add nft transfers to activity
for nft_transfer in nft_transfers:
    nft_id = (nft_transfer['contract_address'], nft_transfer['token_id'])
    date_str = parser.parse(nft_transfer['timestamp']).strftime('%m/%d')
        
    # handle mints
    if nft_transfer['category'] == 'mint':
        nft_activity = 'Minted {}'.format(nft_transfer['quantity'])
    
    # handle burns
    elif nft_transfer['category'] == 'burn':
        nft_activity = 'Burned {}'.format(nft_transfer['quantity'])
        
    # handle sends
    elif nft_transfer['from'] == _target_account:
        nft_activity = 'Sent {} to {}'.format(nft_transfer['quantity'], nft_transfer['to'][:7])
    
    # handle receives
    else:
        nft_activity = 'Received {} from {}'.format(nft_transfer['quantity'], nft_transfer['from'][:7])
    
    # save activity
    if nft_id in nft_activity_map: nft_activity_map[nft_id].append((date_str, nft_activity))
    else: nft_activity_map[nft_id] = [(date_str, nft_activity)]

# add nft sales to activity
for nft_sale in nft_sales:
    nft_id = (nft_sale['contract_address'], nft_sale['token_id'])
    date_str = parser.parse(nft_sale['timestamp']).strftime('%m/%d')
    
    # handle purchases
    if nft_sale['buyer'] == _target_account:
        nft_activity = 'Bought {} for ${} from {}'.format(nft_sale['quantity'], round(nft_sale['usd_price'], 2), nft_sale['seller'][:7])
        
    # handle sales
    else:
        nft_activity = 'Sold {} for ${} to {}'.format(nft_sale['quantity'], round(nft_sale['usd_price'], 2), nft_sale['buyer'][:7])
    
    # save activity
    if nft_id in nft_activity_map: nft_activity_map[nft_id].append((date_str, nft_activity))
    else: nft_activity_map[nft_id] = [(date_str, nft_activity)]
        
# sort nft activity by date
for nft_id in nft_activity_map:
    nft_activity_map[nft_id] = sorted(nft_activity_map[nft_id], key=lambda a: a[0])
    

In [76]:
# STEP 5: display results

print('\nWallet: {}\n\n'.format(_target_account))

# display all owned NFTs
print('=== NFT PORTFOLIO ===\n')
for owned_nft in owned_nfts:
    if owned_nft['name'] is None: name = 'NO_NAME'
    else: name = owned_nft['name']
    print('\t{}: {}'.format(owned_nft['balance'], name))
    
# display all activity for each owned NFT
print('\n\n=== NFT ACTIVITY ===\n')
owned_nfts_map = {(nft['contract_address'], nft['token_id']): nft for nft in owned_nfts}
for nft_id in nft_activity_map:
    
    # retrieve nft in activity
    nft = transpose.nft.nfts_by_token_id(
        contract_addresses=nft_id[0],
        token_ids=nft_id[1]
    ).to_dict()
    
    # print name
    if len(nft) == 0: continue
    print('\t{}'.format(nft[0]['name']))
    
    # print activity
    for date_str, activity in nft_activity_map[nft_id]:
        print('\t\t{}: {}'.format(date_str, activity))
    print()


Wallet: 0x5eeeFc7D872C03296639bcc49D11c99403d2F4A8


=== NFT PORTFOLIO ===

	1: Coniun Pass #1510
	1: Coniun Pass #3756
	1: Zombie Frens Key to the City #180
	1: Zombie Frens Key to the City #181
	1: Zombie Frens Key to the City #182
	1: AO Art Ball #825
	1: Unrevealed Chef
	1: BAYC DP Office Space Cards #480
	3: RTFKT Space Pod
	1: Meebit #162
	2: Em(1)
	1: Bean #16869
	1: Gen-F #5856
	1: Gen-F #6478
	1: Gen-F #8331
	1: You #1015
	1: adidas Originals Meta Capsule
	1: adidas Originals Meta Capsule
	1: Paradise Sky #6
	1: NO_NAME
	1: Call for Peace
	1: Inspiring Teacher
	1: DS-1 Prototype Frigate
	1: goblinchest.wtf Mint Pass
	1: #713
	1: GNSS #146
	1: GNSS #967
	1: GNSS #6947
	1: Minimen #540
	1: RTFKT LOOT Pod
	1: NO_NAME
	1: Atocha
	1: 0407
	1: Rug Radio Membership Pass
	1: NO_NAME
	1: ZombieFren #969
	1: ZombieFren #5075
	1: ZombieFren #5575
	1: ZombieFren #7595
	1: ZombieFren #7596
	1: ZombieFren #8662
	1: iNFT Personality Pod #2414
	1: iNFT Personality Pod #2658
	1: iNFT Personal